In [ ]:
from pathlib import Path
import json

In [ ]:
file = Path("../arc-prize-2025/arc-agi_training_challenges.json")

data = json.load(file.open())

In [ ]:
import numpy as np

for k, v in data.items():
    print("challenge", k)
    # pretty pre
    for example in v["train"]:
        input = example["input"]
        output = example["output"]
        print(np.array(input))
        print(np.array(output))
        break
    break


In [ ]:
from arc_competition.task_viewer import view_training_challenge

# Quick view of a training challenge
view_training_challenge("00576224")

# # Or with custom file path
# from pathlib import Path

# html_view = task_viewer("arc-prize-2025/arc-agi_training_challenges.json", "00576224")
# display(html_view)

In [ ]:
from arc_competition.task_viewer import ARCChallenge, ARCDataLoader, ROOT_DIR

file_path = ROOT_DIR / "arc-prize-2025" / "arc-agi_training_challenges.json"

challenge = ARCDataLoader.load_challenge(file_path, "00576224")

In [ ]:
from arc_competition.agents import create_grid_analysis_agent
from arc_competition.schemas import GridAnalysisInput

# agent = create_grid_analysis_agent()

# user_input = GridAnalysisInput(
#     input_grid=challenge.train[0].input,
#     output_grid=challenge.train[0].output,
# )
# output = agent.run(user_input)


In [ ]:
user_input

In [ ]:
from atomic_agents import AgentConfig, AtomicAgent
from atomic_agents.context import SystemPromptGenerator
from arc_competition.config import config
from pydantic import Field
from atomic_agents.base.base_io_schema import BaseIOSchema
from pydantic import BaseModel
from arc_competition.task_viewer import GridExample
from arc_competition.task_viewer import Grid
from arc_competition.utils import calculate_score, calculate_difference_grid


class GridExampleModifiedInput(GridExample):
    """Extends schema for GridExample."""

    current: Grid = Field(description="Current grid as 2D list of integers")
    difference: Grid = Field(description="Difference grid as 2D list of 1s and 0s")
    score: float = Field(description="Score of the current grid")


# Grid Analysis Schemas
class GridAnalysisInput(BaseIOSchema):
    """Input schema for ARC grid analysis."""

    example_grids: list[GridExampleModifiedInput] = Field(
        description="Example grids containing the input and output grids"
    )


class GridAnalysisOutput(BaseIOSchema):
    """Output schema for ARC grid analysis."""

    # Overall summary
    analysis_summary: str = Field(description="Overall summary of the analysis")

    function_to_execute: str = Field(description="python function to execute")


def create_generator_agent() -> AtomicAgent[GridAnalysisInput, GridAnalysisOutput]:
    """Create a grid pattern analysis agent specialized for ARC challenges."""

    client = config.client
    # Create optimized system prompt generator for grid analysis
    system_prompt_generator = SystemPromptGenerator(
        background=[
            "You are an expert in pattern recognition and visual reasoning, specialized in analyzing grid-based puzzles.",
            "You have deep expertise in ARC (Abstraction and Reasoning Corpus) challenges and can identify subtle patterns.",
            "You understand both low-level pixel patterns and high-level conceptual transformations.",
            "Given an input grid, current grid and an output grid, you can identify the next best function to execute to transform the current grid into the output grid.",
            "Only use small python functions, preferably less than 10 lines of code but no more than 50 lines of code.",
            "You will also be given a score for the current grid, you can use this to identify issues with the functions or the order of execution.",
            "The score is a float between 0 and 100, where 0 is the worst and 100 is the best.",
            "50 points are deducted for mismatch output sizes between the current grid and the output grid.",
            "1 point is deducted for each element in the current grid that is unmatched in the output grid.",
            "all comparisons will be done with the top left element of the current grid as the reference point.",
            "a reference difference grid will be provided to you, you can use this to identify the differences between the current grid and the output grid.",
            "The reference difference grid will be a 2D grid of 0s and 1s, where 0s are the elements that are matched and 1s are the elements that are unmatched.",
        ]
    )

    # Create the grid analysis agent
    grid_agent = AtomicAgent[GridAnalysisInput, GridAnalysisOutput](
        config=AgentConfig(
            client=client,
            model=config.model.name,
            system_prompt_generator=system_prompt_generator,
            model_api_parameters=config.model.get_api_parameters(),
        )
    )

    return grid_agent


def update_to_modified_input(input: Grid, output: Grid) -> GridExampleModifiedInput:
    return GridExampleModifiedInput(
        input=input,
        output=output,
        current=input,
        score=calculate_score(input, output),
        difference=calculate_difference_grid(input, output),
    )


examples = [
    update_to_modified_input(example.input, example.output)
    for example in challenge.train
]

user_input = GridAnalysisInput(
    example_grids=examples,
)

grid_agent = create_generator_agent()
output = grid_agent.run(user_input)

In [ ]:
print(output.function_to_execute)

In [ ]:
def transform_grid(grid):
    # Get the original 2x2 grid
    original = [row[:] for row in grid]

    # Create a flipped version (swap rows)
    flipped = [original[1], original[0]]

    # Create the 6x6 output grid
    output = []

    # First two rows: repeat original horizontally 3 times
    for row in original:
        output.append(row * 3)

    # Next two rows: repeat flipped horizontally 3 times
    for row in flipped:
        output.append(row * 3)

    # Last two rows: repeat original horizontally 3 times again
    for row in original:
        output.append(row * 3)

    return output


llm_output = transform_grid(challenge.train[1].input)
llm_output = Grid(llm_output)
diff_grid = calculate_difference_grid(llm_output, challenge.train[1].output)
calculate_score(llm_output, challenge.train[1].output)

In [ ]:
diff_grid

In [ ]:
challenge.train[0].output

In [ ]:
llm_output

In [ ]:
print(output.function_to_execute)

In [ ]:
print(output.model_dump_json(indent=4))

In [ ]:
print(challenge.train[0].prompt_format())